In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [3]:
pip  install transformers

     |████████████████████████████████| 2.9 MB 5.2 MB/s 
     |████████████████████████████████| 56 kB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 33.8 MB/s 
     |████████████████████████████████| 895 kB 33.1 MB/s 
     |████████████████████████████████| 596 kB 46.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [7]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

import transformers
from transformers import BertTokenizer, RobertaTokenizer
from torch.utils.data import Dataset, DataLoader, TensorDataset
from transformers import BertForSequenceClassification, RobertaForSequenceClassification

BERT_MODEL_NAME = 'bert-base-uncased'
ROBERTA_MODEL_NAME = 'roberta-base'

from nltk.corpus import stopwords 
from collections import Counter
import string
import re
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader

import ast
import random
from collections import OrderedDict
import pickle
import math
import torch.optim as optim
import datetime
import torch.autograd as autograd
from tqdm import tqdm,trange
from sklearn.preprocessing import MultiLabelBinarizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import pandas as pd
import os
import sys
dir_path ='/content/gdrive/My Drive/NLP/Dataset'
sys.path.append(dir_path)

In [8]:
train = pd.read_csv('{}/train.gz'.format(dir_path), index_col = False)
del train['Unnamed: 0']
train['labels'] = train['labels'].apply(lambda x : ast.literal_eval(x))

val = pd.read_csv('{}/val.gz'.format(dir_path), index_col = False)
del val['Unnamed: 0']
val['labels'] = val['labels'].apply(lambda x : ast.literal_eval(x))

In [9]:
train.loc[0,'preprocessed_sentence']

'[CLS] then terrorism struck again , this time in the # indonesia # capital of $ jakarta $ .'

In [10]:
print("Number of sentence with length > 127: {}".format(train[train['preprocessed_sentence'].apply(lambda x: len(x.split()) > 127)].shape[0]))
train = train[train['preprocessed_sentence'].apply(lambda x: len(x.split()) <= 127)]
print(train.shape)

print("Number of sentence with length > 127: {}".format(val[val['preprocessed_sentence'].apply(lambda x: len(x.split()) > 127)].shape[0]))
val = val[val['preprocessed_sentence'].apply(lambda x: len(x.split()) <= 127)]
print(val.shape)

val.loc[0,'preprocessed_sentence']

Number of sentence with length > 127: 144
(146234, 4)
Number of sentence with length > 127: 8
(16266, 4)


'[CLS] mr. scruggs -- who is arguing the case with his son zach ; a colleague from northern $ mississippi $ , don barrett ; and john jones from jackson , miss. -- said he had already filed suit on behalf of about 2,000 clients seeking redress from their insurers , including senator # trent lott # , who is both a neighbor of mr. scruggs in pascagoula and his brother-in-law ; and representative gene taylor of bay st. louis .'

In [11]:
mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform(train['labels'])

In [12]:
train_labels.shape

(146234, 30)

In [13]:
print(train_labels[1])
print(mlb.inverse_transform(train_labels[1].reshape(1,-1)))
print(mlb.classes_)

[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[('/location/administrative_division/country',)]
['/business/company/advisors' '/business/company/founders'
 '/business/company/major_shareholders' '/business/company/place_founded'
 '/business/company_advisor/companies_advised' '/business/person/company'
 '/film/film/featured_film_locations'
 '/film/film_location/featured_in_films'
 '/location/administrative_division/country' '/location/br_state/capital'
 '/location/country/administrative_divisions' '/location/country/capital'
 '/location/location/contains' '/location/neighborhood/neighborhood_of'
 '/location/province/capital' '/location/us_county/county_seat'
 '/location/us_state/capital' '/people/deceased_person/place_of_burial'
 '/people/deceased_person/place_of_death'
 '/people/ethnicity/geographic_distribution' '/people/person/children'
 '/people/person/ethnicity' '/people/person/nationality'
 '/people/person/place_lived' '/people/person/place_of_birth'
 '/people/perso

In [14]:
val_labels = mlb.transform(val['labels'])
val_labels.shape

(16266, 30)

In [15]:
sample_sentence = train.iloc[0]['preprocessed_sentence']
print(sample_sentence)

[CLS] then terrorism struck again , this time in the # indonesia # capital of $ jakarta $ .


In [16]:
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [17]:
tokens = tokenizer.tokenize(sample_sentence)
print(tokens)

['[CLS]', 'then', 'terrorism', 'struck', 'again', ',', 'this', 'time', 'in', 'the', '#', 'indonesia', '#', 'capital', 'of', '$', 'jakarta', '$', '.']


In [18]:
class NYDataset(Dataset):
  def __init__(self, sentences, labels,tokenizer, max_seq_len):
    self.sentences = sentences
    self.labels = labels
    #self.tokenizer = tokenizer
    self.max_seq_len = max_seq_len
    self.cls_token = "[CLS]"
    self.mask_padding_with_zero = True
    self.add_sep_token = False
    self.sequence_a_segment_id = 0
    self.cls_token_segment_id=0
    self.pad_token = 0
    self.pad_token_segment_id=0
  def __len__(self):
    return len(self.sentences)
  def __getitem__(self, item):
    sentence = str(self.sentences[item])
    label = self.labels[item]
    # print(label)
    # print("sentence: ")
    # print(sentence)
    tokens_a = tokenizer.tokenize(sentence)
    # print("tokens: ")
    # print(tokens_a)
    e1_indices = [i for i, letter in enumerate(tokens_a) if letter == '$']
    e11_p = e1_indices[0]  # the start position of entity1
    e12_p = e1_indices[1]  # the end position of entity1
    
    e2_indices = [i for i, letter in enumerate(tokens_a) if letter == '#']
    e21_p = e2_indices[0]  # the start position of entity2
    e22_p = e2_indices[1]  # the end position of entity2

    # Add 1 because of the [CLS] token
    e11_p += 1
    e12_p += 1
    e21_p += 1
    e22_p += 1
    
    #account for cls token
    if self.add_sep_token:
      special_tokens_count = 2
    else:
      special_tokens_count = 1

    if len(tokens_a) > self.max_seq_len - special_tokens_count:
      tokens_a = tokens_a[: (self.max_seq_len - special_tokens_count)]
    
    tokens = tokens_a
    if self.add_sep_token:
        
        tokens += [sep_token]


    token_type_ids = [self.sequence_a_segment_id] * len(tokens)

    tokens = [self.cls_token] + tokens
    token_type_ids = [self.cls_token_segment_id] + token_type_ids

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real tokens are attended to.
    attention_mask = [1 if self.mask_padding_with_zero else 0] * len(input_ids)

    # Zero-pad up to the sequence length.
    padding_length = self.max_seq_len - len(input_ids)
    input_ids = input_ids + ([self.pad_token] * padding_length)
    attention_mask = attention_mask + ([0 if self.mask_padding_with_zero else 1] * padding_length)
    token_type_ids = token_type_ids + ([self.pad_token_segment_id] * padding_length)

    # e1 mask, e2 mask
    e1_mask = [0] * len(attention_mask)
    e2_mask = [0] * len(attention_mask)
    
    for i in range(e11_p, e12_p +1):
        e1_mask[i] = 1
    for i in range(e21_p, e22_p+ 1 ):
        e2_mask[i] = 1
    # encoding = self.tokenizer.encode_plus(
    #   sentence,
    #   add_special_tokens=True,
    #   max_length=self.max_len,
    #   return_token_type_ids=False,
    #   pad_to_max_length=True,
    #   return_attention_mask=True,
    #   return_tensors='pt',
    # )

    assert len(input_ids) == self.max_seq_len, "Error with input length {} vs {}".format(len(input_ids), max_seq_len)
    assert len(attention_mask) == self.max_seq_len, "Error with attention mask length {} vs {}".format(
        len(attention_mask), self.max_seq_len
    )
    assert len(token_type_ids) == self.max_seq_len, "Error with token type length {} vs {}".format(
        len(token_type_ids), self.max_seq_len
    )

    return {
      # 'sentence_text': sentence,
      'input_ids':torch.tensor(input_ids, dtype=torch.long).flatten(),
      'token_type_ids':torch.tensor(token_type_ids, dtype=torch.long),
      'attention_mask': torch.tensor(attention_mask, dtype=torch.long).flatten(),
      'targets': torch.tensor(label, dtype=torch.float),
      'e1_mask':torch.tensor(e1_mask, dtype=torch.long).flatten(),
      'e2_mask':torch.tensor(e2_mask, dtype=torch.long)
    }

In [19]:
def create_data_loader(df,labels,tokenizer, max_len, batch_size):
  train_sampler = RandomSampler(df)
  dataset = NYDataset(
    sentences=df['preprocessed_sentence'].to_numpy(),
    labels=labels,
    tokenizer = tokenizer,
    max_seq_len=max_len
  )
  return DataLoader(
    dataset,
    sampler = train_sampler,
    batch_size=batch_size,
    num_workers=4
  )

In [20]:
import torch
import torch.nn as nn

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

MAX_LENGTH = 128
BATCH_SIZE = 64
train_data_loader = create_data_loader(train[:500],train_labels[:500],tokenizer, MAX_LENGTH, BATCH_SIZE)
val_data_loader = create_data_loader(train[:500],train_labels[:500],tokenizer,MAX_LENGTH,BATCH_SIZE)

from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup

from transformers import BertModel, BertPreTrainedModel

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [21]:
class FCLayer(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_rate=0.0, use_activation=True):
        super(FCLayer, self).__init__()
        self.use_activation = use_activation
        self.dropout = nn.Dropout(dropout_rate)
        self.linear = nn.Linear(input_dim, output_dim)
        self.tanh = nn.Tanh()
        

    def forward(self, x):
        x = self.dropout(x)
        #if self.use_activation:
             #x = self.tanh(x)
        #return self.linear(x)
        x = self.linear(x)
        x = self.tanh(x)
        return x

class SigmoidLayer(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_rate=0.0, use_activation=True):
        super(SigmoidLayer, self).__init__()
        self.use_activation = use_activation
        self.dropout = nn.Dropout(dropout_rate)
        self.linear = nn.Linear(input_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.dropout(x)
        #if self.use_activation:
         #    x = self.sigmoid(x)
        #return self.linear(x)
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

class RBERT(BertPreTrainedModel):
    def __init__(self, config):
        super(RBERT, self).__init__(config)
        self.bert = BertModel(config=config)  # Load pretrained bert

        self.num_labels = config.num_labels

        self.cls_fc_layer = FCLayer(config.hidden_size, config.hidden_size, dropout_rate)
        self.entity_fc_layer = FCLayer(config.hidden_size, config.hidden_size, dropout_rate)
        # self.label_classifier = FCLayer(
        #     config.hidden_size * 3,
        #     config.num_labels,
        #     dropout_rate,
        #     use_activation=False,
        # )
        self.label_classifier = SigmoidLayer(
            config.hidden_size*3,
            config.num_labels,
            dropout_rate,
            use_activation=True,
        )

    @staticmethod
    def entity_average(hidden_output, e_mask):
        """
        Average the entity hidden state vectors (H_i ~ H_j)
        :param hidden_output: [batch_size, j-i+1, dim]
        :param e_mask: [batch_size, max_seq_len]
                e.g. e_mask[0] == [0, 0, 0, 1, 1, 1, 0, 0, ... 0]
        :return: [batch_size, dim]
        """
        e_mask_unsqueeze = e_mask.unsqueeze(1)  # [b, 1, j-i+1]
        length_tensor = (e_mask != 0).sum(dim=1).unsqueeze(1)  # [batch_size, 1]

        # [b, 1, j-i+1] * [b, j-i+1, dim] = [b, 1, dim] -> [b, dim]
        sum_vector = torch.bmm(e_mask_unsqueeze.float(), hidden_output).squeeze(1)
        avg_vector = sum_vector.float() / length_tensor.float()  # broadcasting
        return avg_vector

    def forward(self, input_ids, attention_mask, token_type_ids, labels, e1_mask, e2_mask):
        outputs = self.bert(
            input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids
        )  # sequence_output, pooled_output, (hidden_states), (attentions)
        sequence_output = outputs[0]
        pooled_output = outputs[1]  # [CLS]

        # Average
        e1_h = self.entity_average(sequence_output, e1_mask)
        e2_h = self.entity_average(sequence_output, e2_mask)

        # Dropout -> tanh -> fc_layer (Share FC layer for e1 and e2)
        pooled_output = self.cls_fc_layer(pooled_output)
        e1_h = self.entity_fc_layer(e1_h)
        e2_h = self.entity_fc_layer(e2_h)

        # Concat -> fc_layer
        concat_h = torch.cat([pooled_output, e1_h, e2_h], dim=-1)
        # concat_e = torch.cat([e1_h, e2_h], dim=-1)
        
        # logits = self.label_classifier(e1_h)
        logits = self.label_classifier(concat_h)
        # print(logits)
        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here
        # print(torch.sum(outputs[0]))
        # Softmax
        if labels is not None:
            if self.num_labels == 1:
                loss_fct = nn.MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = nn.BCELoss()
                # print(labels.view(-1, self.num_labels).shape)
                # print(logits.view(-1, self.num_labels).shape )
                # print(labels)
                # print(labels.view(-1, self.num_labels))
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1, self.num_labels))

            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [23]:
config = BertConfig.from_pretrained(
            pretrained_model_name_or_path = 'bert-base-uncased',
            num_labels=30
        )

dropout_rate = 0.3

model = RBERT.from_pretrained(pretrained_model_name_or_path = 'bert-base-uncased',config=config)
# GPU or CPU
# device = "cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu"
model.to(device)

from transformers import AdamW,  get_linear_schedule_with_warmup
EPOCHS = 8
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=2,
  num_training_steps=total_steps
)

device

data = next(iter(train_data_loader))

input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)
token_type_ids = data['token_type_ids'].to(device)
targets = data['targets'].to(device)
e1_mask = data['e1_mask'].to(device)
e2_mask = data['e2_mask'].to(device)

print(input_ids.shape) 
print(attention_mask.shape)
print(token_type_ids.shape) 
print(targets.shape)
print(e1_mask.shape) 
print(e2_mask.shape)

tokenizer.mask_token_id

data['input_ids'][0]

data['e1_mask'][0]

data['e1_mask'][1]

data['input_ids'][1]

data = next(iter(val_data_loader))

input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)
token_type_ids = data['token_type_ids'].to(device)
targets = data['targets'].to(device)
e1_mask = data['e1_mask'].to(device)
e2_mask = data['e2_mask'].to(device)

print(input_ids.shape) 
print(attention_mask.shape)
print(token_type_ids.shape) 
print(targets.shape)
print(e1_mask.shape) 
print(e2_mask.shape)

threshold = 0.5

from sklearn.metrics import classification_report

def evaluate(model,data_loader):
  eval_loss = 0.0
  preds = None
  predictions, true_vals = [], []
  model.eval()
  for batch in tqdm(data_loader,desc="Evaluating",leave=True,position=0):
    
    # batch = tuple(batch[t].to(device) for t in list(batch.keys()))
    batch['input_ids'] = batch['input_ids'].to(device)
    batch['attention_mask']= batch['attention_mask'].to(device)
    batch['token_type_ids'] = batch['token_type_ids'].to(device)
    batch['targets'] = batch['targets'].to(device)
    batch['e1_mask'] = batch['e1_mask'].to(device)
    batch['e2_mask'] = batch['e2_mask'].to(device)

    with torch.no_grad():
      inputs = {
          "input_ids": batch['input_ids'],
          "attention_mask": batch['attention_mask'],
          "token_type_ids": batch['token_type_ids'],
          "labels": batch['targets'],
          "e1_mask": batch['e1_mask'],
          "e2_mask": batch['e2_mask']
      }
      outputs = model(**inputs)
      tmp_eval_loss, logits = outputs[:2]
      eval_loss += tmp_eval_loss.mean().item()
    
   
    if preds is None:
        preds = logits.detach().cpu().numpy()
        out_label_ids = inputs["labels"].detach().cpu().numpy()
    else:
        # temp = logits.detach().cpu().numpy()
        # pred_labels = [np.where(val > threshold,1,0) for val in temp]
        # print(pred_labels)
        preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
        out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)
        
    # break
    # print(preds)
    # print(out_label_ids)
  eval_loss = eval_loss/len(data_loader)
  print("loss: ",eval_loss)
  # print(preds.sum())
  # print(preds)
  preds = [np.where(pred > threshold,1,0) for pred in preds]
  # print(preds)
  # print(out_label_ids)
  print(classification_report(

    out_label_ids,

    preds,

    target_names=mlb.classes_,

    zero_division=0

  ))
  return eval_loss

global_step = 0.0
tr_loss = 0.0
model.zero_grad()

best_model_path =  os.path.join('{}'.format(dir_path), 'Best_model.h5py')

EPOCHS = 4
min_eval_loss = np.inf

train_iterator = trange(int(EPOCHS),desc="Epoch")

for _ in train_iterator:
  
  epoch_iterator = tqdm(train_data_loader, desc='Iteration', position = 0,leave=True, disable=False,)
  for step,batch in enumerate(epoch_iterator):
    model.train()
    # print(step)
    # print(batch.keys())
    # batch = tuple(batch[t].to(device) for t in list(batch.keys()))
    batch['input_ids'] = batch['input_ids'].to(device)
    batch['attention_mask']= batch['attention_mask'].to(device)
    batch['token_type_ids'] = batch['token_type_ids'].to(device)
    batch['targets'] = batch['targets'].to(device)
    batch['e1_mask'] = batch['e1_mask'].to(device)
    batch['e2_mask'] = batch['e2_mask'].to(device)
    inputs = {
        "input_ids": batch['input_ids'],
        "attention_mask": batch['attention_mask'],
        "token_type_ids": batch['token_type_ids'],
        "labels": batch['targets'],
        "e1_mask": batch['e1_mask'],
        "e2_mask": batch['e2_mask']
    }
    outputs = model(**inputs)
    loss = outputs[0]
    loss.backward()
    # _, logits = outputs[:2]
    # print(logits)
    tr_loss += loss.item()

    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    
    optimizer.step()
    scheduler.step()
    # print('training_loss: {:.3f}'.format(loss.item()/len(batch)))
    
    epoch_iterator.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
    # evaluate(model,train_data_loader)
    # break
    # batch_step+=1
  avg_tr_loss = tr_loss/len(train_data_loader)
  tqdm.write(f'Training loss: {avg_tr_loss}')

  eval_loss = evaluate(model,val_data_loader)
  if(eval_loss < min_eval_loss):
    print("Eval loss reduced from {} to {}".format(min_eval_loss,eval_loss))
    print("Saving model")
    min_eval_loss = eval_loss
    torch.save(model.state_dict(),best_model_path)
  # val_loss, predictions, true_vals = evaluate(dataloader_validation)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing RBERT: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing RBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RBERT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['label_classifier.linear.bias', 'entity_fc_layer.

torch.Size([64, 128])
torch.Size([64, 128])
torch.Size([64, 128])
torch.Size([64, 30])
torch.Size([64, 128])
torch.Size([64, 128])
torch.Size([64, 128])
torch.Size([64, 128])
torch.Size([64, 128])
torch.Size([64, 30])
torch.Size([64, 128])
torch.Size([64, 128])


Epoch:   0%|          | 0/4 [00:19<?, ?it/s]

Training loss: 0.44915337301790714


Evaluating: 100%|██████████| 8/8 [00:07<00:00,  1.05it/s]


...

loss:  0.09958226047456264
                                             precision    recall  f1-score   support

                 /business/company/advisors       0.00      0.00      0.00         0
                 /business/company/founders       0.00      0.00      0.00         1
       /business/company/major_shareholders       0.00      0.00      0.00         0
            /business/company/place_founded       0.00      0.00      0.00         1
/business/company_advisor/companies_advised       0.00      0.00      0.00         0
                   /business/person/company       0.00      0.00      0.00        16
         /film/film/featured_film_locations       0.00      0.00      0.00         0
      /film/film_location/featured_in_films       0.00      0.00      0.00         0
  /location/administrative_division/country       0.00      0.00      0.00        18
                 /location/br_state/capital       0.00      0.00      0.00         0
 /location/country/administrative_div

Epoch:  75%|███████▌  | 3/4 [01:46<00:29, 29.15s/it]

Training loss: 0.7865570122376084


Epoch: 100%|██████████| 4/4 [01:54<00:00, 28.59s/it]

loss:  0.10658613964915276
                                             precision    recall  f1-score   support

                 /business/company/advisors       0.00      0.00      0.00         0
                 /business/company/founders       0.00      0.00      0.00         1
       /business/company/major_shareholders       0.00      0.00      0.00         0
            /business/company/place_founded       0.00      0.00      0.00         1
/business/company_advisor/companies_advised       0.00      0.00      0.00         0
                   /business/person/company       0.00      0.00      0.00        16
         /film/film/featured_film_locations       0.00      0.00      0.00         0
      /film/film_location/featured_in_films       0.00      0.00      0.00         0
  /location/administrative_division/country       0.00      0.00      0.00        18
                 /location/br_state/capital       0.00      0.00      0.00         0
 /location/country/administrative_div

In [24]:
#load model
model_path = '{}/Best_model.h5py'.format(dir_path)
model.load_state_dict(torch.load(model_path))

threshold = 0.5
eval_loss = evaluate(model,val_data_loader)

threshold = 0.01
eval_loss = evaluate(model,val_data_loader)

threshold = 0.2
eval_loss = evaluate(model,train_data_loader)

Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Evaluating: 100%|██████████| 8/8 [00:07<00:00,  1.05it/s]


loss:  0.10012712143361568
                                             precision    recall  f1-score   support

                 /business/company/advisors       0.00      0.00      0.00         0
                 /business/company/founders       0.00      0.00      0.00         1
       /business/company/major_shareholders       0.00      0.00      0.00         0
            /business/company/place_founded       0.00      0.00      0.00         1
/business/company_advisor/companies_advised       0.00      0.00      0.00         0
                   /business/person/company       0.00      0.00      0.00        16
         /film/film/featured_film_locations       0.00      0.00      0.00         0
      /film/film_location/featured_in_films       0.00      0.00      0.00         0
  /location/administrative_division/country       0.00      0.00      0.00        18
                 /location/br_state/capital       0.00      0.00      0.00         0
 /location/country/administrative_div

Evaluating: 100%|██████████| 8/8 [00:07<00:00,  1.07it/s]


loss:  0.0997327882796526
                                             precision    recall  f1-score   support

                 /business/company/advisors       0.00      0.00      0.00         0
                 /business/company/founders       0.00      0.00      0.00         1
       /business/company/major_shareholders       0.00      0.00      0.00         0
            /business/company/place_founded       0.00      0.00      0.00         1
/business/company_advisor/companies_advised       0.00      0.00      0.00         0
                   /business/person/company       0.00      0.00      0.00        16
         /film/film/featured_film_locations       0.00      0.00      0.00         0
      /film/film_location/featured_in_films       0.00      0.00      0.00         0
  /location/administrative_division/country       0.00      0.00      0.00        18
                 /location/br_state/capital       0.00      0.00      0.00         0
 /location/country/administrative_divi

Evaluating: 100%|██████████| 8/8 [00:07<00:00,  1.05it/s]

loss:  0.0998626472428441
                                             precision    recall  f1-score   support

                 /business/company/advisors       0.00      0.00      0.00         0
                 /business/company/founders       0.00      0.00      0.00         1
       /business/company/major_shareholders       0.00      0.00      0.00         0
            /business/company/place_founded       0.00      0.00      0.00         1
/business/company_advisor/companies_advised       0.00      0.00      0.00         0
                   /business/person/company       0.00      0.00      0.00        16
         /film/film/featured_film_locations       0.00      0.00      0.00         0
      /film/film_location/featured_in_films       0.00      0.00      0.00         0
  /location/administrative_division/country       0.00      0.00      0.00        18
                 /location/br_state/capital       0.00      0.00      0.00         0
 /location/country/administrative_divi